In [ ]:
# Copyright 2019 Eugene Maslovich
# ehpc@ehpc.io

In [2]:
import time
import math
from mpmath import mp
import mpmath.libmp
mpmath.libmp.BACKEND

'gmpy'

In [5]:
# Проблему замощения прямоугольника AxB прямоугольником 1x2 можно рассматривать как классическую проблему замощения плоскости,
# которая аналитически решается раскрашиванием плоскости шахматной доской. Можно былло копать и дальше в этом направлении, но
# если представить прямоугольник в виде графа, вершины которого - центры раскрашенных квадратов, то проблема сводится к
# нахождению числа совершенных сочетаний планарного графа, т.к. прямоугольник 1x2 является по сути ребром такого графа.
# Существует алгоритм, позволяющий вычислить общую задачу нахождения совершенных сочетаний за полиномиальное время - 
# FKT (Fisher, Kasteleyn, Temperley). В случае нашей задачи, всё сводится к формуле https://books.google.nl/books?id=4pCKDwAAQBAJ&lpg=PA145&ots=8pU1IQa6Ts&dq=Temperley%20Fisher%20Kasteleyn&hl=ru&pg=PA157#v=onepage&q=Temperley%20Fisher%20Kasteleyn&f=false.
# Жуткие синусы и косинусы получаются из вывода детерминанта матрицы алгоритма FKT и описаны в работах Фишера и Кастелейна.
# Для преодоления барьера точности используем модуль mpmath.
# Проверить корректность функции можно сравнением результатов с последовательностью http://oeis.org/A004003.
def pack(a, b):
    if a % 2 != 0 and b %2 != 0:
        return 0
    else:
        p = None
        jtop = math.ceil(a / 2) + 1
        ktop = math.ceil(b / 2) + 1
        ap1 = mp.fadd(a, 1)
        bp1 = mp.fadd(b, 1)
        for j in range(1, jtop):
            for k in range(1, ktop):
                f1cos = mp.cos(mp.fdiv(mp.fmul(mp.pi, mp.mpf(j)), ap1))
                f1 = mp.fmul(4, mp.power(f1cos, 2))
                f2cos = mp.cos(mp.fdiv(mp.fmul(mp.pi, mp.mpf(k)), bp1))
                f2 = mp.fmul(4, mp.power(f2cos, 2))
                f = mp.fadd(f1, f2)
                if p is None:
                    p = f
                else:
                    p = mp.fmul(p, f)
        if p is None:
            return '0'
        else:
            val = mp.nstr(mp.nint(p), mp.dps * 10)
            # Возможна потеря точности, если вернулся False
            if len(val) > mp.dps:
                return [val.split('.')[0], False, len(val)]
            else:
                return [val.split('.')[0], True, len(val)]

In [6]:
# Здесь мы на коленке проверяем корректность алгоритма на известных значениях
def test():
    mp.dps = 100
    n = 10
    # Числа для проверки алгоритма http://oeis.org/A004003
    tests = ['2', '36', '6728', '12988816', '258584046368', '53060477521960000', '112202208776036178000000',
             '2444888770250892795802079170816', '548943583215388338077567813208427340288',
             '1269984011256235834242602753102293934298576249856']
    for a in range(1, n + 1):
        val = pack(2 * a, 2 * a)[0]
        print("pack{}({}, {}) = ".format(a, 2 * a, 2 * a), val, 'WRONG' if val != tests[a - 1] else 'OK')
test()
print("Precision: ".format(), mp)

pack1(2, 2) =  2 OK
pack2(4, 4) =  36 OK
pack3(6, 6) =  6728 OK
pack4(8, 8) =  12988816 OK
pack5(10, 10) =  258584046368 OK
pack6(12, 12) =  53060477521960000 OK
pack7(14, 14) =  112202208776036178000000 OK
pack8(16, 16) =  2444888770250892795802079170816 OK
pack9(18, 18) =  548943583215388338077567813208427340288 OK
pack10(20, 20) =  1269984011256235834242602753102293934298576249856 OK
Precision:  Mpmath settings:
  mp.prec = 336               [default: 53]
  mp.dps = 100                [default: 15]
  mp.trap_complex = False     [default: False]


In [7]:
# Функция вычисления с автоподстройкой точности
def calc_auto(a, b):
    mp.dps = 100
    while True:
        res = pack(a, b)
        if res[1]:
            print("pack({}, {}) = {}. Possibly incorrect: {}. Digits: {}. Precision: {}.".format(a, b, res[0], "false" if res[1] else "true", res[2] - 2, mp.dps))
            break
        mp.dps = res[2] + 100

In [8]:
%%time
calc_auto(20, 30)

pack(20, 30) = 6217539537964764012601603771542270056953148716152633313878015867124216625. Possibly incorrect: false. Digits: 73. Precision: 100.
Wall time: 9.98 ms


In [9]:
%%time
calc_auto(100, 200)

pack(100, 200) = 13184626568252634672850731534369811480035905349663430100982867941140244181342264636212655191058561972818541083395282092787408266067300860524393108721683793545658199102881727357810488930922381918388314128962566699974841113909674378795326966514390085003755502456903073461750529783950391581170209720097316999674839569771093292285529258035999881785318472693384129728244460758715310701060129341244940022622910758667782220078172962770560419228321069896219350502245502614484070894433011171402191171663762697489973334320489969492161323989132854205885326859545536642429515573742532642153773454502116226784822216529312729830223876858114462478008660828669902726402630689380371883529751793493061784469063036543056240058010574591336857866686195407946047020090762279955090850954747680133988697334273785370681840116926747137251427570097280897042067637471675132002385656327778720524637464311897537966998990539016844620824480873967215837728914764297370400718749638166315806611885002074904999830736023

In [ ]:
# Эта функция уже пытается вычислять все возможные значения
def calcmax():
    a = 100
    c = 100
    while True:
        for b in range(a, 2 * a + 1, c):
            %time calc_auto(a, b)
            c = 2 * c
        a = a + c
calcmax()

pack(100, 100) = 21129684951231576218861957857404865711065304360944947214472584397344247961405828599639638701457040244783017711049222591601830662047330989812255303215458007810284845270743993576756196443276430943888987165941442057200120678472089089459372261153374657472665083269766448553006309599596297474055660844036152552088153400929532178494569240491533123478846241821533647951540654789156647731092885629825415386132750603931729973409396477136768730061647795473990770177090389058893977117635680799535862437590663550169511539958091862878312590377243436937439838837303120732581019068314500870431885425512027402475377489365267525605084316141594175280023489255830182477313651425689393806868702909573909828642442875290041574328720979087603688408029227161183236368407792497848934909963834356377543044527428191098398480747195221302860059650024337749629905382778448176632279976441294452965709234228208056601288997974137561452372337710874322848504791116302769536968451401855777820954331544970269676478668324

KeyboardInterrupt: 